차량 데이터가 몇개지? ->7407개

In [4]:
ik="/mnt/hdd_6tb/seungeun/HuNature/data/image/car"
import os
total=0
ikfol = os.listdir(ik)
for i in range(len(ikfol)):
    total+=len(os.listdir(os.path.join(ik,ikfol[i])))
print(total)

7407


번호판은 몇개지? -> 11887개

In [5]:
ik="/mnt/hdd_6tb/seungeun/HuNature/data/image/plate"
import os
total=0
ikfol = os.listdir(ik)
for i in range(len(ikfol)):
    total+=len(os.listdir(os.path.join(ik,ikfol[i])))
print(total)

11887


차량 데이터 불러와서 저장하기 ->7407개 완료=>processed_image

In [1]:
from natsort import natsorted
import os
from PIL import Image
input = "/mnt/hdd_6tb/seungeun/HuNature/data/image/car/"
out = "/mnt/hdd_6tb/bill0914/processed_carcrop/processed_image"
idx = 1
for i in natsorted(os.listdir(input)):
    for k in natsorted(os.listdir(os.path.join(input,i))):
        n = f"car_image_{idx}.png"
        im = Image.open(os.path.join(os.path.join(input,i),k))
        im.save(os.path.join(out,n))
        idx+=1



car데이터 label -> coord만 저장하기 =>processed_label

In [4]:
import os
import json
import natsort
import json
num=1
car_label_dir = "/mnt/hdd_6tb/seungeun/HuNature/data/label/car"
save_dir = "/mnt/hdd_6tb/bill0914/processed_carcrop/processed_label"
os.makedirs(save_dir,exist_ok=True)
for subfolder in natsort.natsorted(os.listdir(car_label_dir)):
    subfolder_path = os.path.join(car_label_dir,subfolder) #car/VL~~~
    for jsons in natsort.natsorted(os.listdir(subfolder_path)):
        jsons_path = os.path.join(subfolder_path,jsons)
        with open(jsons_path,'r') as f:
            label = json.load(f)
            W,H= (label['Raw Data Info']['resolution']).split(',')
            W,H = int(W),int(H)
            label = label["Learning Data Info"]['annotations']
            #print(label[0].get('coord')[0])
            
        for i in range(len(label)): #coord개수만큼 돌리기
            if label[i].get('coord')[0]=='[':
                jj = label[i].get('coord')[1:-1]  
                
                a,b,c,d = map(float,jj.split(','))
                file_name = f"car_image_{num}.txt"
                save_path = os.path.join(save_dir,file_name)
                with open(save_path,'a') as f:
                    if i ==len(label)-1 : 
                        f.write(f"{1} {a} {b} {c} {d}")
                    else:
                        f.write(f"{1} {a} {b} {c} {d}\n")        
            else:                    
                a = label[i].get('coord')[0]
                b = label[i].get('coord')[1]
                c = label[i].get('coord')[2]
                d = label[i].get('coord')[3]
                file_name = f"car_image_{num}.txt"
                save_path = os.path.join(save_dir,file_name)
                with open(save_path,'a') as f:
                    if i ==len(label)-1 : 
                        f.write(f"{1} {a} {b} {c} {d}")
                    else:
                        f.write(f"{1} {a} {b} {c} {d}\n")
        num+=1            
                                           
      


label정규화해서 새로운 폴더에 만들자 =>"/mnt/hdd_6tb/bill0914/processed_carcrop/processed_labelno"
기존 라벨과 다른건 center로 맞춘 뒤 값을 정규화함 

In [2]:
import os
import json
import natsort
import json
num=1
car_label_dir = "/mnt/hdd_6tb/seungeun/HuNature/data/label/car"
save_dir = "/mnt/hdd_6tb/bill0914/carandplate/carlabel"
os.makedirs(save_dir,exist_ok=True)
for subfolder in natsort.natsorted(os.listdir(car_label_dir)):
    subfolder_path = os.path.join(car_label_dir,subfolder) #car/VL~~~
    for jsons in natsort.natsorted(os.listdir(subfolder_path)):
        jsons_path = os.path.join(subfolder_path,jsons)
        with open(jsons_path,'r') as f:
            label = json.load(f)
            W,H= (label['Raw Data Info']['resolution']).split(',')
            W,H = int(W),int(H)
            label = label["Learning Data Info"]['annotations']
            #print(label[0].get('coord')[0])
            
        for i in range(len(label)): #coord개수만큼 돌리기
            if label[i].get('coord')[0]=='[':
                jj = label[i].get('coord')[1:-1]  
                
                a,b,c,d = map(float,jj.split(','))
                file_name = f"car_image_{num}.txt"
                save_path = os.path.join(save_dir,file_name)
                with open(save_path,'a') as f:
                    if i ==len(label)-1 : 
                        f.write(f"{1} {(a+c/2)/W} {(b+d/2)/H} {c/W} {d/H}")
                    else:
                        f.write(f"{1} {(a+c/2)/W} {(b+d/2)/H} {c/W} {d/H}\n")        
            else:                    
                a = label[i].get('coord')[0]
                b = label[i].get('coord')[1]
                c = label[i].get('coord')[2]
                d = label[i].get('coord')[3]
                file_name = f"car_image_{num}.txt"
                save_path = os.path.join(save_dir,file_name)
                with open(save_path,'a') as f:
                    if i ==len(label)-1 : 
                        f.write(f"{1} {(a+c/2)/W} {(b+d/2)/H} {c/W} {d/H}")
                    else:
                        f.write(f"{1} {(a+c/2)/W} {(b+d/2)/H} {c/W} {d/H}\n")
        num+=1            
                                           
      


train/val 3:1로 나눠서 학습용으로 만들기

In [4]:
import os
import shutil
import random

# 원본 디렉토리
label_dir = "/mnt/hdd_6tb/bill0914/processed_original/label"
image_dir = "/mnt/hdd_6tb/bill0914/processed_original/image"

# 복사할 목표 디렉토리
train_label_dir = "/mnt/hdd_6tb/bill0914/carandplate/data/train/labels"
val_label_dir = "/mnt/hdd_6tb/bill0914/carandplate/data/val/labels"
train_image_dir = "/mnt/hdd_6tb/bill0914/carandplate/data/train/images"
val_image_dir = "/mnt/hdd_6tb/bill0914/carandplate/data/val/images"

# 디렉토리 생성 (없으면 생성)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)

# 라벨 파일 목록 가져오기
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# 라벨 파일 섞기 (랜덤 셔플)
random.shuffle(label_files)

# 75%는 train, 25%는 val로 분할
split_index = int(len(label_files) * 0.75)
train_files = label_files[:split_index]
val_files = label_files[split_index:]

# 복사 함수
def copy_files(file_list, src_label_dir, src_image_dir, dest_label_dir, dest_image_dir):
    for label_file in file_list:
        # 라벨 파일 경로
        src_label_path = os.path.join(src_label_dir, label_file)
        dest_label_path = os.path.join(dest_label_dir, label_file)
        
        # 이미지 파일 경로
        image_file = label_file.replace('.txt', '.png')  # 라벨 파일명을 이미지 파일명으로 변환
        src_image_path = os.path.join(src_image_dir, image_file)
        dest_image_path = os.path.join(dest_image_dir, image_file)
        
        # 라벨 복사
        shutil.copy(src_label_path, dest_label_path)
        
        # 이미지가 존재하면 복사
        if os.path.exists(src_image_path):
            shutil.copy(src_image_path, dest_image_path)
        else:
            print(f"이미지 파일 {image_file}가 존재하지 않습니다.")

# Train 데이터 복사
copy_files(train_files, label_dir, image_dir, train_label_dir, train_image_dir)

# Val 데이터 복사
copy_files(val_files, label_dir, image_dir, val_label_dir, val_image_dir)

print(f"Train 라벨 개수: {len(train_files)}")
print(f"Val 라벨 개수: {len(val_files)}")


Train 라벨 개수: 8915
Val 라벨 개수: 2972


이미지에서 차량부분만 떼어보자 - 라벨링 했으니까  

In [ ]:
from PIL import Image
import cv2
import natsort
import os

image_dir = "/mnt/hdd_6tb/bill0914/processed_carcrop/processed_image"
label_dir = "/mnt/hdd_6tb/bill0914/processed_carcrop/processed_label"
out_dir = "/mnt/hdd_6tb/bill0914/processed_carcrop/ori_img"

image_list = natsort.natsorted(os.listdir(image_dir))
label_list = natsort.natsorted(os.listdir(label_dir))

for i, image_name in enumerate(image_list):
    image_path = os.path.join(image_dir, image_name)
    image = cv2.imread(image_path)  # 이미지 불러오기 (array 형태)
    
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        continue
    
    height, width, _ = image.shape  # 이미지의 높이와 너비 가져오기
    
    # 해당 이미지의 라벨 읽기
    label_path = os.path.join(label_dir, label_list[i])
    with open(label_path, 'r') as f:
        carbbox_list = f.readlines()
    
    for idx, g in enumerate(carbbox_list):
        # 라벨의 좌상단 x, y와 w, h 파싱
        class_id, x, y, w, h = map(float, g.split())
        
        # 좌표를 정수로 변환
        x1 = int(x)
        y1 = int(y)
        x2 = int(x + w)
        y2 = int(y + h)
        
        # 이미지 크롭 (좌표가 이미지 경계를 벗어나지 않도록 클램핑)
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(width, x2)
        y2 = min(height, y2)
        
        car_image = image[y1:y2, x1:x2]  # 크롭된 이미지
        
        if car_image.size == 0:
            print(f"크롭된 이미지가 비어 있습니다: {image_name}, 라벨 {idx}")
            continue
        
        # 크롭된 이미지를 PIL 형식으로 변환 후 리사이즈
        car_image = Image.fromarray(cv2.cvtColor(car_image, cv2.COLOR_BGR2RGB))
        car_image = car_image.resize((640, 360))
        
        # 파일 이름 생성 및 저장
        name = f"image_{i + 1}_{idx}.png"
        car_image.save(os.path.join(out_dir, name))
